![image](http://wx2.sinaimg.cn/thumbnail/69d4185bly1fmf9kfagd3j20ek0ekq88.jpg)
# 面向对象高级编程

## 使用__slots__
### 为实例添加属性与方法
正常情况下，当我们定义了一个class，创建了一个class的实例后，我们可以给该实例绑定任何属性和方法，这就是动态语言的灵活性。

In [5]:
# 创建一个类
class Student(object):
    pass

s = Student()

# 为实例绑定属性
s.name = 'Michael'
print(s.name)

# 为实例绑定方法
# 定义一个函数作为实例方法，同时为实例绑定属性
def set_age(self, age): 
    self.age = age
    
from types import MethodType
s.set_age = MethodType(set_age, s) # 给实例绑定一个方法
s.set_age(25) # 调用实例方法
print(s.age)

Michael
25


### 为class添加属性与方法
但是，给一个实例绑定的方法，对另一个实例是不起作用的，为了给所有实例都绑定方法，可以给class绑定方法。给class绑定方法后，所有实例均可调用。

In [7]:
# s2 = Student() # 创建新的实例
# s2.set_age(25) # 尝试调用方法

def set_score(self, score):
    self.score = score
# 为class绑定方法
Student.set_score = set_score

# 所有实例均可调用
s.set_score(100)
print(s.score)

s2.set_score(99)
print(s2.score)

100
99


### 使用__slots__显示属性的个数
当我们只允许对Student实例添加name和age属性时，就可以使用`__slots__`属性限制。

In [8]:
class Student(object):
    __slots__ = ('name', 'age') # 用tuple定义允许绑定的属性名称
    
s = Student() # 创建新的实例
s.name = 'Michael' # 绑定属性'name'
s.age = 25 # 绑定属性'age'

# name、age之外的属性绑定会报错
s.score = 99 # 绑定属性'score'

AttributeError: 'Student' object has no attribute 'score'

由于`'score'`没有被放到`__slots__`中，所以不能绑定`score`属性，试图绑定`score`将得到`AttributeError`的错误。

使用`__slots__`要注意，`__slots__`定义的属性仅对**当前类实例**起作用，对继承的**子类**是不起作用的。除非在子类中也定义`__slots__`，这样，子类实例允许定义的属性就是自身的`__slots__`加上父类的`__slots__`。

In [9]:
class GraduateStudent(Student):
    pass

g = GraduateStudent()
# 子类实例可以绑定age、name之外的属性，不受父类 __slots__的限制
g.score = 9999

## 使用@property

### 使用get与set方法设置和获取属性
在绑定属性时，如果我们直接把属性暴露出去，虽然写起来很简单，但是，没办法检查参数，导致可以把成绩随便改。这显然不合逻辑。为了限制score的范围，可以通过一个`set_score()`方法来设置成绩，再通过一个`get_score()`来获取成绩，这样，在`set_score()`方法里，就可以检查参数。

In [11]:
class Student(object):

    def get_score(self):
         return self._score

    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value
        
# score 属性就不可以随便设置
s = Student()
s.set_score(60) # ok!
print(s.get_score())

s.set_score(9999)

60


ValueError: score must between 0 ~ 100!

### 使用@property将方法转成属性调用

上面属性的设置较为麻烦，python内置的 `@property` 装饰器就是负责把一个方法变成属性调用。

In [13]:
class Student(object):

    @property
    def score(self):
        return self._score

    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

`@property`的实现比较复杂，我们先考察如何使用。把一个`getter`方法变成属性，只需要加上`@property`就可以了，此时，`@property`本身又创建了另一个装饰器`@score.setter`，负责把一个`setter`方法变成属性赋值，于是，我们就拥有一个可控的属性操作。

In [14]:
s = Student()
s.score = 60 # OK，实际转化为s.set_score(60)
print(s.score) # OK，实际转化为s.get_score()

s.score = 9999

60


ValueError: score must between 0 ~ 100!

### 设置只读属性
只定义getter方法，不定义setter方法就是一个只读属性，如下，`birth`是可读写属性，而`age`就是一个只读属性，因为`age`可以根据`birth`和当前时间计算出来。

In [15]:
class Student(object):

    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self, value):
        self._birth = value

    @property
    def age(self):
        return 2015 - self._birth

###  练习
利用`@property`给一个`Screen`对象加上`width`和`height`属性，以及一个只读属性`resolution`。

In [18]:
class Screen(object):
    @property
    def width(self):
        return self._width
    @width.setter
    def width(self, value):
        self._width = value
        
    @property
    def height(self):
        return self._height
    @height.setter
    def height(self, value):
        self._height = value
    
    @property
    def resolution(self):
        return self._width * self._height

# 测试:
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过!')
else:
    print('测试失败!')

resolution = 786432
测试通过!


## 多重继承

### 类层次的划分
通过继承，子类就可以扩展父类的功能。对于类的继承结构，不同的划分方法具有不同结构。对于`Animal`类，要对如下几个动物进行划分
```
Dog - 狗狗；
Bat - 蝙蝠；
Parrot - 鹦鹉；
Ostrich - 鸵鸟。
```
我们可以按照哺乳动物和鸟类归类设计类的层次结构，也可以按照“能跑”和“能飞”来归类。
![image](http://wx3.sinaimg.cn/large/69d4185bly1fn236l4bf5j20kz078weg.jpg)

如果要把上面的两种分类都包含进来，我们就得设计更多的层次：
- 哺乳类：能跑的哺乳类，能飞的哺乳类；
- 鸟类：能跑的鸟类，能飞的鸟类。
这么一来，类的层次就复杂了。
![image](http://wx3.sinaimg.cn/large/69d4185bly1fn2376v6ujj20ba08vjr9.jpg)

如果要再增加“宠物类”和“非宠物类”，这么搞下去，类的数量会呈指数增长，很明显这样设计是不行的。

正确的做法是采用多重继承。首先，主要的类层次仍按照哺乳类和鸟类设计：

In [22]:
class Animal(object):
    pass

# 大类:
class Mammal(Animal):
    pass

class Bird(Animal):
    pass

# 各种动物:
class Dog(Mammal):
    pass

class Bat(Mammal):
    pass

class Parrot(Bird):
    pass

class Ostrich(Bird):
    pass

现在，我们要给动物再加上`Runnable`和`Flyable`的功能，只需要先定义好`Runnable`和`Flyable`的类

In [19]:
class Runnable(object):
    def run(self):
        print('Running...')

class Flyable(object):
    def fly(self):
        print('Flying...')

对于需要`Runnable`功能的动物，就多继承一个`Runnable`，例如`Dog`；对于需要`Flyable`功能的动物，就多继承一个`Flyable`，例如`Bat`。通过多重继承，一个子类就可以同时获得多个父类的所有功能。

In [ ]:
class Dog(Mammal, Runnable):
    pass

class Bat(Mammal, Flyable):
    pass

### MixIn
在设计类的继承关系时，通常，主线都是单一继承下来的，例如，`Ostrich`继承自`Bird`。但是，如果需要“混入”额外的功能，通过多重继承就可以实现，比如，让`Ostrich`除了继承自`Bird`外，再同时继承`Runnable`。这种设计通常称之为`MixIn`。

为了更好地看出继承关系，我们把`Runnable`和`Flyable`改为`RunnableMixIn`和`FlyableMixIn`。类似的，你还可以定义出肉食动物`CarnivorousMixIn`和植食动物`HerbivoresMixIn`，让某个动物同时拥有好几个`MixIn`。

In [25]:
class RunnableMixIn(object):
    pass

class CarnivorousMixIn(object):
    pass

class Dog(Mammal, RunnableMixIn, CarnivorousMixIn):
    pass

MixIn的目的就是给一个类增加多个功能，这样，在设计类的时候，我们优先考虑通过多重继承来组合多个MixIn的功能，而不是设计多层次的复杂的继承关系。

Python自带的很多库也使用了MixIn。举个例子，Python自带了`TCPServer`和`UDPServer`这两类网络服务，而要同时服务多个用户就必须使用多进程或多线程模型，这两种模型由`ForkingMixIn`和`ThreadingMixIn`提供。通过组合，我们就可以创造出合适的服务来。

In [ ]:
# 编写一个多进程模式的TCP服务
class MyTCPServer(TCPServer, ForkingMixIn):
    pass

# 编写一个多线程模式的UDP服务
class MyUDPServer(UDPServer, ThreadingMixIn):
    pass

# 更先进的协程模型，可以编写一个CoroutineMixIn
class MyTCPServer(TCPServer, CoroutineMixIn):
    pass

## 定制类
看到类似`__slots__`这种形如`__xxx__`的变量或者函数名就要注意，这些在Python中是有特殊用途的。

`__slots__`我们已经知道怎么用了，`__len__()`方法我们也知道是为了能让`class`作用于`len()`函数。

除此之外，Python的class中还有许多这样有特殊用途的函数，可以帮助我们定制类。详细的特殊函数使用可以参考 Python 官网 [Special method names](https://docs.python.org/3/reference/datamodel.html#special-method-names)

### \__str\__
`__str__()`方法可以用于定制一个格式化的打印参数。如在但打印类的时候，通常是把类在内存中的地址打印出来，在类中添加 `__str__()` 可以定制打印的内容，也使得打印出的内容更易于理解。

In [27]:
class Student(object):
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return 'Student object (name: %s)' % self.name

s = Student('Michael')
print(s)
s

Student object (name: Michael)


从上面可以看到，不适用`print`，直接调用实例打印的还是之前的内容。

这是因为直接显示变量调用的不是`__str__()`，而是`__repr__()`，两者的区别是`__str__()`返回用户看到的字符串，而`__repr__()`返回程序开发者看到的字符串，也就是说，`__repr__()`是为调试服务的。

解决办法是再定义一个`__repr__()`。但是通常`__str__()`和`__repr__()`代码都是一样的，所以，有个偷懒的写法：

In [28]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name=%s)' % self.name
    __repr__ = __str__
    
s = Student('Michael')
print(s)
s

Student object (name=Michael)


Student object (name=Michael)

###  \__iter\__

如果一个类想被用于`for ... in`循环，类似`list`或`tuple`那样，就必须实现一个`__iter__()`方法，该方法返回一个迭代对象，然后，Python的`for`循环就会不断调用该迭代对象的`__next__()`方法拿到循环的下一个值，直到遇到`StopIteration`错误时退出循环。

以斐波那契数列为例，写一个Fib类，可以作用于for循环：

In [29]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1 # 初始化两个计数器a，b

    def __iter__(self):
        return self # 实例本身就是迭代对象，故返回自己

    def __next__(self):
        self.a, self.b = self.b, self.a + self.b # 计算下一个值
        if self.a > 4: # 退出循环的条件
            raise StopIteration()
        return self.a # 返回下一个值

# 将 for in 用于 fib 类
for n in Fib():
    print(n)

1
1
2
3


### \__getitem\__

Fib实例虽然能作用于for循环，看起来和list有点像，但是，把它当成list来使用还是不行，比如，取第5个元素。要表现得像list那样按照下标取出元素，需要实现`__getitem__()`方法。

In [31]:
Fib()[5]

TypeError: 'Fib' object does not support indexing

In [32]:
class Fib(object):
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a + b
        return a
f = Fib()
print(f[1])
print(f[2])
print(f[3])
print(f[7])

1
2
3
21


单 list 的切片法此时并不能用，原因是`__getitem__()`传入的参数可能是一个`int`，也可能是一个切片对象`slice`，所以要做判断：

In [34]:
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int): # n是索引
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice): # n是切片
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a, b = b, a + b
            return L

# 在 Fib 的实例对象上调用切片方法
f = Fib()
print(f[0:5])
print(f[:10])

# 没有对步长进行处理
print(f[:10:2])

[1, 1, 2, 3, 5]
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


也没有对负数作处理，所以，要正确实现一个`__getitem__()`还是有很多工作要做的。

此外，如果把对象看成`dict`，`__getitem__()`的参数也可能是一个可以作`key`的`object`，例如`str`。

与之对应的是`__setitem__()`方法，把对象视作`list`或`dict`来对集合赋值。最后，还有一个`__delitem__()`方法，用于删除某个元素。

总之，通过上面的方法，我们自己定义的类表现得和Python自带的`list`、`tuple`、`dict`没什么区别，这完全归功于动态语言的“鸭子类型”，不需要强制继承某个接口。

### \__getattr\__
正常情况下，当我们调用类的方法或属性时，如果不存在，就会报错。比如定义`Student`类，调用`name`属性，没问题，但是，调用不存在的`score`属性，就有问题了。

In [38]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'

s = Student()
print(s.name)
# 报错
print(s.score)

Michael


AttributeError: 'Student' object has no attribute 'score'

错误信息很清楚地告诉我们，没有找到`score`这个`attribute`。

要避免这个错误，除了可以加上一个`score`属性外，Python还有另一个机制，那就是写一个`__getattr__()`方法，动态返回一个属性。

当调用不存在的属性时，比如`score`，Python解释器会试图调用`__getattr__(self, 'score')`来尝试获得属性，这样，我们就有机会返回score的值。

In [39]:
class Student(object):

    def __init__(self):
        self.name = 'Michael'

    def __getattr__(self, attr):
        if attr=='score':
            return 99

s = Student()
print(s.name)
print(s.score)

Michael
99


返回函数也是完全可以的，调用的时候要使用调用函数的方式。

In [42]:
class Student(object):

    def __getattr__(self, attr):
        if attr=='age':
            return lambda: 25
s = Student()
print(s.age())

25


注意，只有在没有找到属性的情况下，才调用`__getattr__`，已有的属性，比如`name`，不会在`__getattr__`中查找。

此外，注意到任意调用如`s.abc`都会返回`None`，这是因为我们定义的`__getattr__`默认返回就是`None`。要让class只响应特定的几个属性，我们就要按照约定，抛出`AttributeError`的错误：

In [43]:
class Student(object):

    def __getattr__(self, attr):
        if attr=='age':
            return lambda: 25
        # 未响应到的抛出错误
        raise AttributeError('\'Student\' object has no attribute \'%s\'' % attr)

这实际上可以把一个类的所有属性和方法调用全部动态化处理了，不需要任何特殊手段。

这种完全动态调用的特性有什么实际作用呢？作用就是，可以针对完全动态的情况作调用。

举个例子：现在很多网站都搞REST API，比如新浪微博、豆瓣啥的，调用API的URL类似：

- http://api.server/user/friends
- http://api.server/user/timeline/list

如果要写SDK，给每个URL对应的API都写一个方法，那得累死，而且，API一旦改动，SDK也要改。

利用完全动态的`__getattr__`，我们可以写出一个链式调用：

In [44]:
class Chain(object):

    def __init__(self, path=''):
        self._path = path

    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path

    __repr__ = __str__

Chain().user.get.userinfo.list

/user/get/userinfo/list

### \__call\__
一个对象实例可以有自己的属性和方法，当我们调用实例方法时，我们用`instance.method()`来调用。能不能直接在实例本身上调用呢？在Python中，答案是肯定的。

任何类，只需要定义一个`__call__()`方法，就可以直接对实例进行调用。

In [45]:
class Student(object):
    def __init__(self, name):
        self.name = name

    def __call__(self):
        print('My name is %s.' % self.name)
        
s = Student('Michael')
# 在实例本上调用方法
s()

My name is Michael.


`__call__()`还可以定义参数。对实例进行直接调用就好比对一个函数进行调用一样，所以你完全可以把对象看成函数，把函数看成对象，因为这两者之间本来就没啥根本的区别。

如果你把对象看成函数，那么函数本身其实也可以在运行期动态创建出来，因为类的实例都是运行期创建出来的，这么一来，我们就模糊了对象和函数的界限。

那么，怎么判断一个变量是对象还是函数呢？其实，更多的时候，我们需要判断一个对象是否能被调用，能被调用的对象就是一个`Callable`对象，比如函数和我们上面定义的带有`__call__()`的类实例：

In [47]:
print(callable(Student("hello")))
print(callable(max))
print(callable([1, 2, 3]))
print(callable(None))
print(callable('str'))

True
True
False
False
False


## 使用枚举类
当我们需要定义常量时，一个办法是用大写变量通过整数来定义，例如月份：
```
JAN = 1
FEB = 2
MAR = 3
...
NOV = 11
DEC = 12
```
好处是简单，缺点是类型是`int`，并且仍然是变量。

更好的方法是为这样的枚举类型定义一个class类型，然后，每个常量都是class的一个唯一实例。Python提供了`Enum`类来实现这个功能。

这样我们就获得了`Month`类型的枚举类，可以直接使用`Month.Jan`来引用一个常量，或者枚举它的所有成员。

In [48]:
from enum import Enum

Month = Enum('Month', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

for name, member in Month.__members__.items():
    print(name, '=>', member, ',', member.value)

Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
Apr => Month.Apr , 4
May => Month.May , 5
Jun => Month.Jun , 6
Jul => Month.Jul , 7
Aug => Month.Aug , 8
Sep => Month.Sep , 9
Oct => Month.Oct , 10
Nov => Month.Nov , 11
Dec => Month.Dec , 12


`value`属性则是自动赋给成员的`int`常量，默认从1开始计数。

如果需要更精确地控制枚举类型，可以从`Enum`派生出自定义类：

In [49]:
from enum import Enum, unique

# @unique装饰器可以帮助我们检查保证没有重复值。
@unique
class Weekday(Enum):
    Sun = 0 # Sun的value被设定为0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6

访问这些枚举类型可以有若干种方法：

In [50]:
day1 = Weekday.Mon
print(day1)
print(Weekday['Mon'])
print(Weekday.Mon.value)
print(day1 == Weekday.Mon)
print(day1 == Weekday.Tue)
print(Weekday(1))
print(day1 == Weekday(1))
for name, member in Weekday.__members__.items():
    print(name, '=>', member)
    
Weekday(7)

Weekday.Mon
Weekday.Mon
1
True
False
Weekday.Mon
True
Sun => Weekday.Sun
Mon => Weekday.Mon
Tue => Weekday.Tue
Wed => Weekday.Wed
Thu => Weekday.Thu
Fri => Weekday.Fri
Sat => Weekday.Sat


ValueError: 7 is not a valid Weekday

既可以用成员名称引用枚举常量，又可以直接根据value的值获得枚举常量

### 练习

把`Student`的`gender`属性改造为枚举类型，可以避免使用字符串：

In [52]:
from enum import Enum, unique

@unique
class Gender(Enum):
    Male = 0
    Female = 1

class Student(object):
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender
        
# 测试:
bart = Student('Bart', Gender.Male)
if bart.gender == Gender.Male:
    print('测试通过!')
else:
    print('测试失败!')

测试通过!


## 使用元类
### type()
动态语言和静态语言最大的不同，就是函数和类的定义，不是编译时定义的，而是运行时动态创建的。比方说我们要定义一个`Hello`的`class`，就写一个`hello.py`模块：
```python
class Hello(object):
    def hello(self, name='world'):
        print('Hello, %s.' % name)
```
当Python解释器载入`hello`模块时，就会依次执行该模块的所有语句，执行结果就是动态创建出一个`Hello`的class对象，测试如下：
```python
from hello import Hello
h = Hello()
h.hello() # 输出 Hello, world.

type(Hello) # 输出 <class 'type'>

type(h) #输出 <class 'hello.Hello'>
```
`type()`函数可以查看一个类型或变量的类型，`Hello`是一个class，它的类型就是`type`，而`h`是一个实例，它的类型就是class `Hello`。

我们说class的定义是运行时动态创建的，而创建class的方法就是使用`type()`函数。

`type()`函数既可以返回一个对象的类型，又可以创建出新的类型，比如，我们可以通过`type()`函数创建出`Hello`类，而无需通过`class Hello(object)...`的定义：

In [1]:
def fn(self, name='world'): # 先定义函数
    print('Hello, %s.' % name)

Hello = type('Hello', (object,), dict(hello=fn)) # 创建Hello class

# 创建一个实例
h = Hello()
h.hello() # 调用实例中的方法
print(type(Hello)) # <class 'type'>
print(type(h)) # <class '__main__.Hello'>

Hello, world.
<class 'type'>
<class '__main__.Hello'>


要创建一个class对象，`type()`函数依次传入3个参数：

- class的名称；
- 继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；
- class的方法名称与函数绑定，这里我们把函数`fn`绑定到方法名`hello`上。

通过`type()`函数创建的类和直接写class是完全一样的，因为Python解释器遇到class定义时，仅仅是扫描一下class定义的语法，然后调用`type()`函数创建出class。

正常情况下，我们都用`class Xxx...`来定义类，但是，`type()`函数也允许我们动态创建出类来，也就是说，动态语言本身支持运行期动态创建类，这和静态语言有非常大的不同，要在静态语言运行期创建类，必须构造源代码字符串再调用编译器，或者借助一些工具生成字节码实现，本质上都是动态编译，会非常复杂。

### metaclass
除了使用`type()`动态创建类以外，要控制类的创建行为，还可以使用metaclass。metaclass，直译为元类，简单的解释就是：

当我们定义了类以后，就可以根据这个类创建出实例，所以：*先定义类，然后创建实例*。但是如果我们想创建出类呢？那就必须根据metaclass创建出类，所以：*先定义metaclass，然后创建类*。

连接起来就是：**先定义metaclass，就可以创建类，最后创建实例**。

所以，metaclass允许你创建类或者修改类。换句话说，你可以把类看成是metaclass创建出来的“实例”。metaclass是Python面向对象里最难理解，也是最难使用的魔术代码。正常情况下，你不会碰到需要使用metaclass的情况，所以，以下内容看不懂也没关系，因为基本上你不会用到。

我们先看一个简单的例子，这个metaclass可以给我们自定义的MyList增加一个`add`方法：
定义`ListMetaclass`，按照默认习惯，metaclass的类名总是以Metaclass结尾，以便清楚地表示这是一个metaclass：

In [2]:
# metaclass是类的模板，所以必须从`type`类型派生：
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

有了ListMetaclass，我们在定义类的时候还要指示使用ListMetaclass来定制类，传入关键字参数`metaclass`。

当我们传入关键字参数`metaclass`时，魔术就生效了，它指示Python解释器在创建MyList时，要通过`ListMetaclass.__new__()`来创建，在此，我们可以修改类的定义，比如，加上新的方法，然后，返回修改后的定义。

`__new__()`方法接收到的参数依次是：

- 当前准备创建的类的对象；
- 类的名字；
- 类继承的父类集合；
- 类的方法集合。

In [6]:
class MyList(list, metaclass=ListMetaclass):
    pass

# 测试 MyList 的 add方法
my = MyList()
my.add(1)
# add 方法生效
print(my)

# 普通的 list 没有 add 方法
L2 = list()
L2.add(2)

[1]


AttributeError: 'list' object has no attribute 'add'

动态修改有什么意义？直接在`MyList`定义中写上`add()`方法不是更简单吗？正常情况下，确实应该直接写，通过metaclass修改纯属变态。

但是，总会遇到需要通过metaclass修改类定义的。ORM就是一个典型的例子。

ORM全称“Object Relational Mapping”，即对象-关系映射，就是把关系数据库的一行映射为一个对象，也就是一个类对应一个表，这样，写代码更简单，不用直接操作SQL语句。

要编写一个ORM框架，所有的类都只能动态定义，因为只有使用者才能根据表的结构定义出对应的类来。

让我们来尝试编写一个ORM框架。

编写底层模块的第一步，就是先把调用接口写出来。比如，使用者如果使用这个ORM框架，想定义一个`User`类来操作对应的数据库表`User`，我们期待他写出这样的代码：

In [11]:
# 需要先运行后续的代码在运行此处的代码
class User(Model):
    # 定义类的属性到列的映射：
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')

# 创建一个实例：
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
# 保存到数据库：
u.save()

{'__module__': '__main__', '__qualname__': 'User', 'id': <__main__.IntegerField object at 0x0000024495EA19B0>, 'name': <__main__.StringField object at 0x0000024495EA1A20>, 'email': <__main__.StringField object at 0x0000024495EA1A58>, 'password': <__main__.StringField object at 0x0000024495EA1A90>}
Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:username>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
SQL: insert into User (id,username,email,password) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']


其中，父类`Model`和属性类型`StringField`、`IntegerField`是由ORM框架提供的，剩下的魔术方法比如`save()`全部由metaclass自动完成。虽然metaclass的编写会比较复杂，但ORM的使用者用起来却异常简单。

现在，我们就按上面的接口来实现该ORM。

首先来定义Field类，它负责保存数据库表的字段名和字段类型：

In [9]:
class Field(object):

    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type

    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)

在`Field`的基础上，进一步定义各种类型的`Field`，比如`StringField`，`ntegerField`等等：

In [10]:
class StringField(Field):
    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')

class IntegerField(Field):
    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')

下一步，就是编写最复杂的`ModelMetaclass`了，以及基类`Model`：

In [7]:
class ModelMetaclass(type):

    def __new__(cls, name, bases, attrs):
        print(attrs)
        if name=='Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings # 保存属性和列的映射关系
        attrs['__table__'] = name # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)

class Model(dict, metaclass=ModelMetaclass):

    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))    


{'__module__': '__main__', '__qualname__': 'Model', '__init__': <function Model.__init__ at 0x0000024495E8F6A8>, '__getattr__': <function Model.__getattr__ at 0x0000024495E8F048>, '__setattr__': <function Model.__setattr__ at 0x0000024495E8F158>, 'save': <function Model.save at 0x0000024495E8F488>, '__classcell__': <cell at 0x0000024495D8DF18: empty>}


当用户定义一个`class User(Model)`时，Python解释器首先在当前类`User`的定义中查找`metaclass`，如果没有找到，就继续在父类`Model`中查找`metaclass`，找到了，就使用`Model`中定义的`metaclass`的`ModelMetaclass`来创建`User`类，也就是说，`metaclass`可以隐式地继承到子类，但子类自己却感觉不到。

在ModelMetaclass中，一共做了几件事情：
- 排除掉对Model类的修改；
- 在当前类（比如`User`）中查找定义的类的所有属性，如果找到一个`Field`属性，就把它保存到一个`__mappings__`的dict中，同时从类属性中删除该`Field`属性，否则，容易造成运行时错误（实例的属性会遮盖类的同名属性）；
- 把表名保存到`__table__`中，这里简化为表名默认为类名。

在`Model`类中，就可以定义各种操作数据库的方法，比如`save()`，`delete()`，`find()`，`update`等等。

我们实现了`save()`方法，把一个实例保存到数据库中。因为有表名，属性到字段的映射和属性值的集合，就可以构造出INSERT语句。

## 参考
1、[使用__slots__ - 廖雪峰](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/00143186739713011a09b63dcbd42cc87f907a778b3ac73000)

2、[使用@property - 廖雪峰](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/00143186781871161bc8d6497004764b398401a401d4cce000)

3、[多重继承 - 廖雪峰](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014318680104044a55f4a9dbf8452caf71e8dc68b75a18000)

4、[定制类 - 廖雪峰](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014319098638265527beb24f7840aa97de564ccc7f20f6000)

5、[使用枚举类 - 廖雪峰](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/00143191235886950998592cd3e426e91687cdae696e64b000)

6、[使用元类 - 廖雪峰](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014319106919344c4ef8b1e04c48778bb45796e0335839000)
